In [3]:
import numpy as np
import matplotlib as plt

In [4]:
pi = np.pi
nan = np.nan

In [5]:
u0 = 0
uf = 2*pi

v0 = 0
vf = pi

n = 20 # quadrature points in theta
m = 20 # quadrature points in phi

a = 1 # ellipsoid radii
b = 2
c = 3

h = (uf - u0) / n
k = (vf - v0) / m

# These get used in greens and are temporary
DynVis = 0.5 # Dynamic Viscosity arbitrarily chosen. This should be nondimensionalized, or I should 
GreensCnst = 1 / ( 8 * pi * DynVis ) # look up a good value, but honestly I just can't be bothered

In [6]:
mu = np.zeros( (m+1) * (n+1) )

In [7]:
## coordinate defs

def crt2sph(x):
    # Enter cartesan coordinates as x = [x0,x1,x2] 
    # and receive spherical coordinates as r = [rho,theta,phi]
    rho = np.sqrt( x[0]**2 + x[1]**2 + x[2]**2 )
    theta = np.arctan( x[0] / x[1] )
    phi = np.arctan( ( x[0]**2 + x[1]**2 ) / x[2] )
    return [rho, theta, phi]

def sph2crt(r):
    # Enter spherical coordinates as r = [rho,theta,phi]
    # and receive cartesan coordinates as x = [x0,x1,x2] 
    x0 = r[0] * np.cos(r[1]) * np.sin(r[2])
    x1 = r[0] * np.sin(r[1]) * np.sin(r[2])
    x2 = r[0] * np.cos(r[2])
    return [x0, x1, x2]

def bdy(u,v):
    # Enter parameters for the boundary of an ellipse as (theta,phi)
    # and receive cartesan coordinates [x0,x1,x2]
    return [a*np.cos(u)*np.sin(v), b*sin(u)*sin(v), c*cos(v)]

def rho(u,v):
    # Enter the parameters for the boundary of an ellipse as (theta,phi)
    # and receive distance to origin from boundary at that point 
    return np.sqrt( bdy(u,v)[0]**2 + bdy(u,v)[1]**2 + bdy(u,v)[2]**2 )

def rhoify(u,v):
    # Enter the parameters for the boundary of an ellipse as (theta,phi)
    # and receive spherical coordinates as r = [rho,theta,phi] for a point on the boundary
    return [rho(u,v),u,v]
                       
## function defs

def greens(x):
    # Enter cartesan vector as x = [a, b, c] 
    # and receive the Greens for Stokes, aka the Oseen Tensor 
    xn = np.linalg.norm(x, 2)
    A = np.matmul(xn, np.identity(3))
    B = np.matmul(xn**3, np.outer(x, x))
    result = np.matmul( GreensCnst, A + B )
    return result

def jacobian(u,v):
    # Enter the parameters for the boundary of an ellipse as (theta,phi)
    # and receive the jacobian evaluated there
    return a * b * c * np.sin(v) * np.sqrt( (b * c)**2 * np.cos(u)**2 * np.sin(v)**2 + \
                                           (a * c)**2 * np.sin(u)**2 * np.sin(v)**2 + \
                                           (a * b)**2 * np.cos(v)**2 )